In [89]:
# Import packages
from bs4 import BeautifulSoup
import requests
from time import sleep
from datetime import datetime
import csv
import pandas as pd
import json

In [90]:
# Create a function to collect all page urls

# But first create a base_url and a counter variable
counter = 1
base_url = "https://www.mediamarkt.nl/nl/category/_smartphones-483222.html?searchParams=&sort=&view=&page="
# Now create the function
def generate_page_urls(base_url, num_pages):
    page_urls = []
    
    for counter in range (1, num_pages + 1):
        full_url = base_url + str(counter)
        page_urls.append(full_url)
    
    return page_urls

print("Done!")

Done!


In [92]:
# Use the first function to define
#1 Create base_urls for the various categories
base_url_smartphones = "https://www.mediamarkt.nl/nl/category/_smartphones-483222.html?searchParams=&sort=&view=&page="
base_url_laptops = "https://www.mediamarkt.nl/nl/category/_laptops-482723.html?searchParams=&sort=&view=&page="
base_url_tablets = "https://www.mediamarkt.nl/nl/category/_tablets-645048.html?searchParams=&sort=&view=&page="
base_url_tvs = "https://www.mediamarkt.nl/nl/category/_televisies-450682.html?searchParams=&sort=&view=&page="
page_urls = generate_page_urls(base_url_smartphones, 1) + generate_page_urls(base_url_laptops, 1) + generate_page_urls(base_url_tablets, 1) + generate_page_urls(base_url_tvs, 1)
page_urls

['https://www.mediamarkt.nl/nl/category/_smartphones-483222.html?searchParams=&sort=&view=&page=1',
 'https://www.mediamarkt.nl/nl/category/_laptops-482723.html?searchParams=&sort=&view=&page=1',
 'https://www.mediamarkt.nl/nl/category/_tablets-645048.html?searchParams=&sort=&view=&page=1',
 'https://www.mediamarkt.nl/nl/category/_televisies-450682.html?searchParams=&sort=&view=&page=1']

In [93]:
# Create a function to collect the product_urls
def create_product_urls(page_urls):
    product_urls = []
    for page_url in page_urls:
        res = requests.get(page_url)
        soup = BeautifulSoup(res.text, "html.parser")
        products = soup.find_all("h2")
        
        for product in products:
            try:
                product_url = "https://www.mediamarkt.nl" + product.find("a").attrs["href"]
                product_urls.append(product_url)
            except:
                print("dit is geen product")
            
        sleep(5)
        
    return product_urls

print("Done!")

Done!


In [94]:
# Use the create_product_urls function to create a list product_urls
product_urls = create_product_urls(page_urls)
product_urls

dit is geen product
dit is geen product
dit is geen product
dit is geen product
dit is geen product
dit is geen product
dit is geen product
dit is geen product
dit is geen product
dit is geen product
dit is geen product
dit is geen product
dit is geen product


['https://www.mediamarkt.nl/nl/product/_motorola-moto-e7-power-64-gb-dual-sim-blauw-1714397.html',
 'https://www.mediamarkt.nl/nl/product/_samsung-galaxy-a32-5g-128-gb-zwart-1686726.html',
 'https://www.mediamarkt.nl/nl/product/_samsung-galaxy-a52-128-gb-zwart-1686730.html',
 'https://www.mediamarkt.nl/nl/product/_motorola-moto-g60s-128gb-dual-sim-blauw-1705142.html',
 'https://www.mediamarkt.nl/nl/product/_samsung-galaxy-a22-5g-64-gb-grijs-1698326.html',
 'https://www.mediamarkt.nl/nl/product/_samsung-galaxy-s21-5g-128-gb-grijs-1684429.html',
 'https://www.mediamarkt.nl/nl/product/_apple-iphone-11-64-gb-zwart-1677771.html',
 'https://www.mediamarkt.nl/nl/product/_samsung-galaxy-z-fold3-5g-256-gb%C2%A0-zwart-1702961.html',
 'https://www.mediamarkt.nl/nl/product/_samsung-galaxy-a12-32-gb-zwart-1702095.html',
 'https://www.mediamarkt.nl/nl/product/_samsung-galaxy-a12-32-gb-wit-1702096.html',
 'https://www.mediamarkt.nl/nl/product/_samsung-galaxy-a32-4g-128-gb-zwart-1686722.html',
 'https

In [95]:
# Search for the right elements, store them in variables an put them together in a dictionary
product_data = []

def scrape(product_urls):
    for url in product_urls:
        res = requests.get(url)
        soup = BeautifulSoup(res.text, "html.parser") 
        device_type = soup.find(class_ = "breadcrumbs").find_all("li")[2].text.replace("\n", "")
        names = soup.find(class_ = "stickable").img.attrs["alt"]
        try:
            prices = soup.find("div", class_ = "price big").text
        except:
            prices = "no price"
        instock = soup.find(class_ = "box infobox availability").meta.attrs["content"]
        try:
            ratings = soup.find(class_ = "bvseo-ratingValue").text
        except:
            ratings = "no rating"
        try: 
            reviews = soup.find(class_ = "bvseo-reviewCount").text
        except: 
            reviews = "no reviews"
    
        # Get product attributes and store them in attributes_json
        attributes = soup.find(class_ = "specification").find_all('dt')
        values = soup.find(class_ = "specification").find_all('dd')

        attributes_json = {}
        for x, y in zip(attributes, values):
            attributes_json[x.text]=y.text
    
        # Store all variables in products
        products = {"device_type": device_type, "name": names, "price": prices, "instock": instock, "rating": ratings, 
                    "nr_reviews": reviews, "attributes": attributes_json}
        product_data.append(products)
        print("Saving: ", products["name"])
    
    return(product_data)

In [96]:
product_data = scrape(product_urls)

Saving:  MOTOROLA moto e7 power - 64 GB Dual-Sim Blauw
Saving:  SAMSUNG Galaxy A32 5G - 128 GB Zwart
Saving:  SAMSUNG Galaxy A52 - 128 GB Zwart
Saving:  MOTOROLA moto g60s - 128GB Dual-Sim - Blauw
Saving:  SAMSUNG Galaxy A22 5G - 64 GB Grijs
Saving:  SAMSUNG Galaxy S21 5G - 128 GB Grijs
Saving:  APPLE iPhone 11 - 64 GB Zwart
Saving:  SAMSUNG Galaxy Z Fold3 5G - 256 GB  Zwart
Saving:  SAMSUNG Galaxy A12 - 32 GB Zwart
Saving:  SAMSUNG Galaxy A12 - 32 GB Wit
Saving:  SAMSUNG Galaxy A32 4G - 128 GB Zwart
Saving:  MOTOROLA moto e20 - 32GB Dual-Sim - Grijs
Saving:  SAMSUNG Galaxy A03s - 32 GB Zwart
Saving:  XIAOMI Redmi 9A
Saving:  SAMSUNG Galaxy A52s 5G - 128 GB Zwart
Saving:  XIAOMI POCO X3 Pro - 256 GB Blauw
Saving:  APPLE iPhone 11 - 64 GB Wit
Saving:  SAMSUNG Galaxy Xcover 5 EE - 64 GB Zwart
Saving:  SAMSUNG Galaxy A52 - 128 GB Blauw
Saving:  APPLE iPhone 12 - 128 GB Zwart 5G
Saving:  MOTOROLA moto g60 - 128 GB Dual-Sim Grijs
Saving:  SAMSUNG Galaxy S20 FE - 128 GB Donkerblauw 5G
Saving

In [97]:
# Write the results to a csv_file
with open("scraper_team7V2.csv", "w", encoding = 'utf-8', newline = '') as csv_file: 
    writer = csv.writer(csv_file, delimiter = ";")
    writer.writerow(["device_type", "name", "price", "instock", "rating", "nr_reviews", "attributes", "date_and_time"])
    now = datetime.now()
    for product in product_data: 
        writer.writerow([product["device_type"], product["name"], product["price"], product["instock"], product["rating"], 
                         product["nr_reviews"], product["attributes"], now])
print('Done!')

Done!
